In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
checkpoints_path = "/Volumes/well_logs_nrt/autoloader/checkpoints_volume/all_wells_gold_checkpoints"
output_path = "/Volumes/well_logs_nrt/gold/deltalake_folders_volume/all_wells_gold"
input_path = "/Volumes/well_logs_nrt/silver/deltalake_folders_volume/all_wells_silver"

df = spark.readStream.format("delta").load(input_path)

# Data cleaning

df_cleaned = (df.filter(col("well_id").isNotNull()) 
            # .filter(col("DEPTH") >= 0)
            # Normaliza sentinel de profundidad por si acaso (puedes quitarlo si ya viene como NULL)
            #.withColumn("DEPTH", when(col("DEPTH").cast("double") == lit(-999.25), lit(None))
                          #.otherwise(col("DEPTH").cast("double")))
            .dropDuplicates(["well_id", "measurement_time"])
            .withColumn("gold_ingestion_time", current_timestamp())
            .drop(col("silver_ingestion_time"), col("_rescued_data"))
)

#Data validations

df_validated = df_cleaned.withColumn(
    "quality_check",
    when((col("DEPTH") >= 0), "valid")
    .otherwise("Not passed")
)

df_validated.writeStream.format("delta") \
  .partitionBy("well_id") \
  .option("checkpointLocation", checkpoints_path) \
  .option("mergeSchema", "true") \
  .outputMode("append") \
  .trigger(once=True) \
  .start(output_path)

In [0]:
display(spark.read.format("delta").load("/Volumes/well_logs_nrt/gold/deltalake_folders_volume/all_wells_gold")
        .where("well_id = 'F02-01_logs'")
        #.orderBy(col("well_id").asc(), col("depth").asc())
        .orderBy("well_id", "DEPTH", ascending = [True, True])
)